In [1]:
import requests
import base64
import os
import mimetypes
from PIL import Image
from datetime import datetime

In [ ]:
# Azure OpenAI API Configuration
AZURE_OPENAI_API_KEY="YOUR_API_KEY"
AZURE_OPENAI_ENDPOINT="https://YOUR_RESOURCE_NAME.openai.azure.com/"
AZURE_OPENAI_API_VERSION="2025-04-01-preview"
AZURE_OPENAI_IMAGE_DEPLOYMENT="gpt-image-1"

In [3]:
def generate_image(prompt):
    url = f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{AZURE_OPENAI_IMAGE_DEPLOYMENT}/images/generations?api-version={AZURE_OPENAI_API_VERSION}"
    headers = {
        'api-key': AZURE_OPENAI_API_KEY,
        'Content-Type': 'application/json'
    }
    
    data = {
        "prompt": "富士山と桜の美しい風景を写術風に描いてください",
        "n": 1,
        "size": "1024x1024",
        "quality": "medium",
        "output_format": "png"
    }
    
    response = requests.post(url, headers=headers, json=data)
    print("response code: " + str(response.status_code))

    if response.status_code == 200:
        b64 = response.json()["data"][0]["b64_json"]
        output_filename = "./generated/image/" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".png"
        with open(output_filename, "wb") as out:
            out.write(base64.b64decode(b64))
        print("Image saved to: " + output_filename)
    else:
        print("Error: " + response.text)
        exit(1)

In [4]:
generate_image("富士山と桜の美しい風景を写術風に描いてください")

response code: 200
Image saved to: ./generated/image/20250814_153841.png


In [ ]:
def edit_image(prompt,images,generated_image):
    url = f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{AZURE_OPENAI_IMAGE_DEPLOYMENT}/images/edits?api-version={AZURE_OPENAI_API_VERSION}"
    headers = {
        "api-key": AZURE_OPENAI_API_KEY
    }
    
    data = {
        "prompt": prompt,
        "size": "1024x1024",
        "n": 1
    }

    files = []
    for image in images:
        if not os.path.exists(image):
            print(f"Error: Image file {image} does not exist.")
            exit(1)
        filename = os.path.basename(image)
        filetype, _ = mimetypes.guess_type(image)
        files.append(("image[]", (filename, open(image, "rb"), filetype)))
    
    response = requests.post(url, headers=headers, data=data, files=files)
    print("response code: " + str(response.status_code))

    if response.status_code == 200:
        b64 = response.json()["data"][0]["b64_json"]
        with open(generated_image, "wb") as out:
            out.write(base64.b64decode(b64))
        print("Image saved to: " + generated_image)
    else:
        print("Error: " + response.text)
        exit(1)